In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
LAdata = pd.read_csv('la_neighborhoods final.csv')

In [4]:
LAdata[0:5]

,Name,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759884,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539


In [5]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The LA are {}, {}.'.format(latitude, longitude))

The LA are 34.0536909, -118.2427666.


In [6]:
# create map of New York using latitude and longitude values
map_LA = folium.Map(location=[latitude, longitude], zoom_start=10)


In [7]:
# add markers to map
for lat, lng, name in zip(LAdata['Latitude'], LAdata['Longitude'], LAdata['Name']):
    label = '{}, {}'.format(name,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  

In [8]:
map_LA

In [9]:
CLIENT_ID = 'ZFOKV0CS1ATVUOVYRYL2CQLGBX4LEXS5NSHRBYIHREF2E1TC' # your Foursquare ID
CLIENT_SECRET = 'DHXZD352KCYQR5IRBJWN33JUNUXY5KS3MX3HLBZIFCR0BNHP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZFOKV0CS1ATVUOVYRYL2CQLGBX4LEXS5NSHRBYIHREF2E1TC
CLIENT_SECRET:DHXZD352KCYQR5IRBJWN33JUNUXY5KS3MX3HLBZIFCR0BNHP


In [10]:
neighborhood_latitude = latitude
neighborhood_longitude = longitude

In [11]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZFOKV0CS1ATVUOVYRYL2CQLGBX4LEXS5NSHRBYIHREF2E1TC&client_secret=DHXZD352KCYQR5IRBJWN33JUNUXY5KS3MX3HLBZIFCR0BNHP&v=20180605&ll=34.0536909,-118.2427666&radius=500&limit=100'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa770081c176430f1611e50'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Civic Center',
  'headerFullLocation': 'Civic Center, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 34.0581909045, 'lng': -118.23734531946405},
   'sw': {'lat': 34.0491908955, 'lng': -118.24818788053594}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fecf601067d351381ea64fa',
       'name': 'Grand Park',
       'location': {'address': '200 N Grand Ave',
        'crossStreet': 'btwn Temple & 1st St',
        'lat': 34.05503441823839,
        'lng': -118.245

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Grand Park,Park,34.055034,-118.245179
1,Badmaash,Indian Restaurant,34.051342,-118.244571
2,Redbird,American Restaurant,34.050666,-118.244068
3,Kinokuniya Bookstore,Bookstore,34.050145,-118.242246
4,JiST Cafe,Breakfast Spot,34.050908,-118.240436


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

28 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
LA_venues = getNearbyVenues(names=LAdata['Name'],
                                   latitudes=LAdata['Latitude'],
                                   longitudes=LAdata['Longitude']
                                  )

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

In [18]:
print(LA_venues.shape)
LA_venues.head()

(2955, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355,-118.169810,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355,-118.169810,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461,-118.300208,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461,-118.300208,Shell,34.033095,-118.300025,Gas Station
4,Adams-Normandie,34.031461,-118.300208,Little Xian,34.032292,-118.299465,Sushi Restaurant


In [19]:
LA_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton,2,2,2,2,2,2
Adams-Normandie,9,9,9,9,9,9
Agoura Hills,27,27,27,27,27,27
Agua Dulce,1,1,1,1,1,1
Alhambra,14,14,14,14,14,14
Alondra Park,3,3,3,3,3,3
Altadena,5,5,5,5,5,5
Arcadia,24,24,24,24,24,24
Arleta,5,5,5,5,5,5


In [20]:
print('There are {} uniques categories.'.format(len(LA_venues['Venue Category'].unique())))

There are 317 uniques categories.


In [21]:
#Now analyze each neighborhood 

# one hot encoding
LA_onehot = pd.get_dummies(LA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
LA_onehot['Neighborhood'] = LA_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [LA_onehot.columns[-1]] + list(LA_onehot.columns[:-1])
LA_onehot = LA_onehot[fixed_columns]

LA_onehot.head()






,Yoga Studio,ATM,Accessories Store,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,State / Provincial Park,Steakhouse,

In [22]:
LA_onehot.shape

(2955, 317)

In [23]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category 
LA_grouped = LA_onehot.groupby('Neighborhood').mean().reset_index()



In [24]:
LA_grouped.shape

(233, 317)

In [25]:
##get the top 5 

num_top_venues = 5

for hood in LA_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = LA_grouped[LA_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
###

----Acton----
                        venue  freq
0  Construction & Landscaping   1.0
1                 Yoga Studio   0.0
2                      Office   0.0
3            Pedestrian Plaza   0.0
4                        Park   0.0


----Adams-Normandie----
              venue  freq
0  Sushi Restaurant  0.33
1              Park  0.11
2     Grocery Store  0.11
3      Home Service  0.11
4        Playground  0.11


----Agoura Hills----
                  venue  freq
0  Fast Food Restaurant  0.15
1        Breakfast Spot  0.07
2    Chinese Restaurant  0.07
3           Gas Station  0.04
4               Brewery  0.04


----Agua Dulce----
                           venue  freq
0                        Airport   1.0
1                    Yoga Studio   0.0
2                         Notary   0.0
3                           Park   0.0
4  Paper / Office Supplies Store   0.0


----Alhambra----
                 venue  freq
0    Convenience Store  0.29
1          Pizza Place  0.07
2       Hardware Store  

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = LA_grouped['Neighborhood']

for ind in np.arange(LA_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LA_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot


In [28]:
###Cluster Neighborhoods 
# set number of clusters
kclusters = 5

LA_grouped_clustering = LA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 2, 2, 3, 0, 2, 2, 2, 2], dtype=int32)

In [29]:
neighborhoods_venues_sorted.head()

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [30]:
LA_merged = LAdata

In [31]:
LA_merged.head()

,Name,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759884,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539


KeyError: "['Neighborhood'] not found in axis"

In [33]:
LA_merged = LA_merged.rename(columns={'Name': 'Neighborhood'})
LA_merged.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759884,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539


In [34]:

LA_merged = LA_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

LA_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4.0,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2.0,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2.0,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3.0,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot


In [35]:
LA_merged.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4.0,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2.0,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2.0,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3.0,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot


In [38]:
LA_merged.dtypes

Neighborhood               object
Longitude                 float64
Latitude                  float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [48]:
LA_merged

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4.0,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2.0,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2.0,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3.0,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot
5,Alondra Park,-118.335156,33.889617,0.0,Home Service,Baseball Field,Park,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
6,Artesia,-118.080101,33.866896,2.0,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Boutique,Frozen Yogurt Shop
7,Altadena,-118.136239,34.193871,2.0,Food,Bakery,Campground,Notary,Pharmacy,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
8,Angeles Crest,-117.922395,34.313937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Arcadia,-118.030419,34.133230,2.0,Tea Room,Cosmetics Shop,Convenience Store,Gym,Bakery,Golf Course,Football Stadium,Thai Restaurant,Mexican Restaurant,Japanese Restaurant


In [59]:
LA_merged = LA_merged.dropna()

In [60]:
LA_merged.dtypes

Neighborhood               object
Longitude                 float64
Latitude                  float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [61]:
LA_merged[0:10]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4.0,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2.0,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2.0,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3.0,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot
5,Alondra Park,-118.335156,33.889617,0.0,Home Service,Baseball Field,Park,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
6,Artesia,-118.080101,33.866896,2.0,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Boutique,Frozen Yogurt Shop
7,Altadena,-118.136239,34.193871,2.0,Food,Bakery,Campground,Notary,Pharmacy,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,-118.030419,34.133230,2.0,Tea Room,Cosmetics Shop,Convenience Store,Gym,Bakery,Golf Course,Football Stadium,Thai Restaurant,Mexican Restaurant,Japanese Restaurant
10,Arleta,-118.430757,34.243100,2.0,Bakery,Home Service,Historic Site,Convenience Store,Video Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


In [63]:
LA_merged["Cluster Labels"].astype(int)
LA_merged[0:10]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4.0,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2.0,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2.0,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3.0,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot
5,Alondra Park,-118.335156,33.889617,0.0,Home Service,Baseball Field,Park,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
6,Artesia,-118.080101,33.866896,2.0,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Boutique,Frozen Yogurt Shop
7,Altadena,-118.136239,34.193871,2.0,Food,Bakery,Campground,Notary,Pharmacy,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,-118.030419,34.133230,2.0,Tea Room,Cosmetics Shop,Convenience Store,Gym,Bakery,Golf Course,Football Stadium,Thai Restaurant,Mexican Restaurant,Japanese Restaurant
10,Arleta,-118.430757,34.243100,2.0,Bakery,Home Service,Historic Site,Convenience Store,Video Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


In [71]:
LA_merged["Cluster Labels"].astype(int)

0      4
1      2
2      2
3      2
4      3
5      0
6      2
7      2
9      2
10     2
11     2
13     2
14     3
15     3
16     2
17     2
18     0
19     3
20     2
21     2
22     3
23     2
24     2
26     2
27     3
28     2
29     2
30     2
31     2
32     2
33     2
34     2
36     3
38     2
40     0
42     3
43     2
44     2
45     2
47     3
48     2
49     2
50     2
52     2
53     2
54     2
55     3
56     2
57     2
58     2
59     2
60     2
61     3
63     2
64     2
65     2
67     2
68     3
69     2
71     2
72     3
73     2
74     2
75     2
76     2
77     2
78     3
79     2
80     2
81     2
82     2
83     2
84     2
85     2
86     3
87     2
88     2
89     2
90     2
91     1
92     2
93     3
94     4
95     2
96     0
97     2
99     3
100    2
101    2
102    0
103    3
104    4
105    3
106    3
107    2
109    2
110    2
111    3
112    2
113    2
114    2
115    0
116    2
117    2
118    4
119    3
120    1
121    2
122    0
123    2
124    2
1

In [72]:
LA_merged.astype({'Cluster Labels': 'int'}).dtypes

Neighborhood               object
Longitude                 float64
Latitude                  float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [73]:
LA_merged[0:10]

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,4,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
1,Adams-Normandie,-118.300208,34.031461,2,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,-118.759884,34.146736,2,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,-118.317104,34.504927,2,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
4,Alhambra,-118.136512,34.085539,3,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot
5,Alondra Park,-118.335156,33.889617,0,Home Service,Baseball Field,Park,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
6,Artesia,-118.080101,33.866896,2,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Boutique,Frozen Yogurt Shop
7,Altadena,-118.136239,34.193871,2,Food,Bakery,Campground,Notary,Pharmacy,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,-118.030419,34.133230,2,Tea Room,Cosmetics Shop,Convenience Store,Gym,Bakery,Golf Course,Football Stadium,Thai Restaurant,Mexican Restaurant,Japanese Restaurant
10,Arleta,-118.430757,34.243100,2,Bakery,Home Service,Historic Site,Convenience Store,Video Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [80]:
####Examine Clusters
LA_merged.loc[LA_merged['Cluster Labels'] == 0, LA_merged.columns[[0] + list(range(4, LA_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Alondra Park,Home Service,Baseball Field,Park,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
18,Baldwin Hills/Crenshaw,Park,Food,Music Venue,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
40,Chatsworth,Park,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Women's Store
96,Hacienda Heights,Park,Scenic Lookout,Women's Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
102,Harvard Park,Park,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Women's Store
115,Huntington Park,Park,Burger Joint,Fried Chicken Joint,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
122,La Crescenta-Montrose,Park,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Women's Store
135,La Verne,Pool,Park,Women's Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
155,Mission Hills,Park,Church,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
160,Montecito Heights,Lake,Park,Food,Flower Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop


In [81]:
LA_merged.loc[LA_merged['Cluster Labels'] == 1, LA_merged.columns[[0] + list(range(4, LA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Glendale,Trail,Pharmacy,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
120,Industry,Racetrack,Trail,Arts & Entertainment,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
161,Mount Washington,Trail,Garden,Park,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
172,Palos Verdes Estates,Convenience Store,Gas Station,Trail,Women's Store,Farm,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
191,Rancho Palos Verdes,Trail,Women's Store,Electronics Store,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
211,Sierra Madre,Trail,Other Repair Shop,Gourmet Shop,Coffee Shop,Women's Store,Electronics Store,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant
235,Tujunga,Trail,Lake,Women's Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop


In [82]:
LA_merged.loc[LA_merged['Cluster Labels'] == 2, LA_merged.columns[[0] + list(range(4, LA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Adams-Normandie,Sushi Restaurant,Park,Gas Station,Home Service,Playground,Taco Place,Grocery Store,Fabric Shop,Empanada Restaurant,English Restaurant
2,Agoura Hills,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gas Station,Sushi Restaurant,Liquor Store,Bakery,Lounge,BBQ Joint,Shipping Store
3,Agua Dulce,Airport,Women's Store,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
6,Artesia,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Boutique,Frozen Yogurt Shop
7,Altadena,Food,Bakery,Campground,Notary,Pharmacy,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,Tea Room,Cosmetics Shop,Convenience Store,Gym,Bakery,Golf Course,Football Stadium,Thai Restaurant,Mexican Restaurant,Japanese Restaurant
10,Arleta,Bakery,Home Service,Historic Site,Convenience Store,Video Store,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room
11,Arlington Heights,Korean Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Latin American Restaurant,Donut Shop,Karaoke Bar,Sushi Restaurant,Liquor Store,Fried Chicken Joint,Food
13,Atwater Village,Taco Place,Food Court,Coffee Shop,Mobile Phone Shop,Mexican Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Shopping Mall,Fried Chicken Joint,Electronics Store
16,Azusa,Thrift / Vintage Store,Liquor Store,Big Box Store,Pharmacy,Pizza Place,Coffee Shop,Farm,Entertainment Service,Escape Room,Ethiopian Restaurant


In [84]:
LA_merged.loc[LA_merged['Cluster Labels'] == 3, LA_merged.columns[[0] + list(range(4, LA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Alhambra,Convenience Store,Sporting Goods Shop,Fast Food Restaurant,Pet Store,Business Service,Mexican Restaurant,Video Store,Hardware Store,Pizza Place,Breakfast Spot
14,Avalon,Golf Course,Mexican Restaurant,American Restaurant,Playground,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop
15,Avocado Heights,Mexican Restaurant,Fast Food Restaurant,Bakery,Taco Place,Diner,Discount Store,Pizza Place,Liquor Store,Fried Chicken Joint,Grocery Store
19,Baldwin Park,Thrift / Vintage Store,Mexican Restaurant,Liquor Store,Business Service,Latin American Restaurant,Italian Restaurant,Women's Store,Farm,Entertainment Service,Escape Room
22,Bell Gardens,Mexican Restaurant,Park,Grocery Store,ATM,Seafood Restaurant,Burger Joint,Convenience Store,Men's Store,Fast Food Restaurant,Fried Chicken Joint
27,Burbank,Mexican Restaurant,Spa,Rental Service,Asian Restaurant,Food Truck,Fast Food Restaurant,Concert Hall,Tanning Salon,Warehouse Store,Deli / Bodega
36,Canoga Park,Mexican Restaurant,Sports Bar,Ice Cream Shop,Liquor Store,Restaurant,Sushi Restaurant,Farmers Market,Entertainment Service,Escape Room,Ethiopian Restaurant
42,Central-Alameda,Park,Mexican Restaurant,Liquor Store,Donut Shop,Farm,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
47,Chesterfield Square,Burger Joint,Marijuana Dispensary,Mexican Restaurant,Hardware Store,Mediterranean Restaurant,Grocery Store,Farm,Entertainment Service,Escape Room,Ethiopian Restaurant
55,Cypress Park,Park,Bakery,Mexican Restaurant,Discount Store,Farmers Market,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant


In [85]:
LA_merged.loc[LA_merged['Cluster Labels'] == 4, LA_merged.columns[[0] + list(range(4, LA_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
94,Granada Hills,Home Service,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
104,Hasley Canyon,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
118,Irwindale,Business Service,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Women's Store
128,Lake Los Angeles,Construction & Landscaping,Women's Store,Farmers Market,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
156,Monterey Park,Business Service,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Women's Store
221,South San Jose Hills,Business Service,Candy Store,Home Service,Construction & Landscaping,Farmers Market,Entertainment Service,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
